<a href="https://colab.research.google.com/github/DylanCTY/TextAnalytics_LearningSpace/blob/main/IB9CW0_5504008_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Ollama

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  44063      0 --:--:-- --:--:-- --:--:-- 44116
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Get Ollama running in the background of the Colab

In [ ]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 1807
ollama version is 0.1.39


Query the model and get it to generate some text about Game of Thrones?

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Briefly introduce the football games in the UK from 2004 to 2005."

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   1% ▕

Start to build our RAG. Firstly, this involves installing a bunch of packages:

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  

With everything installed we can setup (in Python) our LLM model:

In [ ]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

Read in the text and implement the SetnenceSplitter

In [ ]:
# Ensure the necessary packages are installed
!pip install llama-index llama-index-embeddings-huggingface

from zipfile import ZipFile
import os
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

# Unzip the dataset
zip_file_path = '/content/bbcsport_football.zip'  # Path to the uploaded zip file
unzip_dir = '/content/bbcsport_football'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Combine the content of all text files into a single string, excluding hidden and system files
combined_text = ""
for root, dirs, files in os.walk(unzip_dir):
    for filename in sorted(files):
        if filename.endswith(".txt") and not filename.startswith('._'):
            with open(os.path.join(root, filename), 'r', encoding='utf-8', errors='ignore') as file:
                combined_text += file.read() + "\n"

# Write the combined text to a temporary file
temp_file_path = '/content/combined_bbcsport.txt'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    temp_file.write(combined_text)

# Read the combined text to verify its content
with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
    content = temp_file.read()
    print(f"Combined text length: {len(content)} characters")
    print(f"First 1000 characters of the combined text:\n{content[:1000]}")

# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize the SentenceSplitter with desired settings
parser = SentenceSplitter(chunk_size=200, chunk_overlap=100)
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")

Combined text length: 519713 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup win

Wayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.

Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.

Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take compl

In [ ]:
# Access the first node's text
print(bbcsport_nodes[1].text)

Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.

Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Goodison Park pitch that was cutting up. Fortune gave United the lead after 23 minutes, rising to meet Ronaldo's cross from eight yards after the Portuguese youngster had been allowed too much time and space by the hapless Gary Naysmith.


Check the output

With all our documents created, we can check the first one to make sure everything worked:

(Pending)Now we will use Llama Index's SimpleDirectoryReader to load all the documents. After this we will specify an embedding model from HuggingFace and add all of this to Llama Index's settings:

(Pending)Now we will setup a vector database (this time using Chroma DB) and index our documents (via embeddings). As in the previous tutorial, this will allow us to search for relevant documents to our query:

In [ ]:
# Use the combined text file from the previous steps
combined_file_path = "/content/combined_bbcsport.txt"
reader = FlatReader()
docs = reader.load_data(Path(combined_file_path))
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 1 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("demo-for-bbcsport-football")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='demo-for-bbcsport-football' id=UUID('90754433-0aee-4489-9f15-3a294cd12ad4') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-bbcsport-football


(Pending) Now everything is set up we can start (retrieval augmented) generating! Let's try a simple query about LLMs:

In [ ]:
# Initialize query engine and run sample queries
query_engine = index.as_query_engine()  # use the vector db for queries

# Sample queries for testing
queries = [
    "Who won the Premier League in 2005?",
    "Describe the key players in Manchester United during the 2004-2005 season.",
    "Highlight the top goal scorers in the Premier League for the 2004-2005 season.",
    "How did Chelsea perform in the 2004-2005 football season?",
    "Provide an overview of the UEFA Champions League matches involving UK teams in 2004-2005.",
    "Detail the performance of Liverpool in the 2004-2005 season."
]

# Execute and print responses for each query
for query in queries:
    response = query_engine.query(query)  # query the model with context
    print(f"Query: {query}")
    print(f"Response: {response.response}\n")

Query: Who won the Premier League in 2005?
Response:  As an AI language model, I do not have access to any real-time data or information. However, based on the provided context information, we can infer that Manchester United won the Premier League in 2005.


Query: Describe the key players in Manchester United during the 2004-2005 season.
Response:  Based on the given context, it is mentioned that Manchester United's top-of-the-league status was under threat when they lost a game against Bolton. The top goal scorers of the league were Ryan Giggs and Cristiano Ronaldo, who were both fined for their part in an incident involving Arsenal'r Paul


Query: Highlight the top goal scorers in the Premier League for the 2004-2005 season.
Response:  I do not have the ability to access real-time data or statistics. However, according to the given context, one can assume that Wayne Rooney and Thierry Henry are likely to be the top goal scorers based on their mentioned achievements in the text.


Q

In [ ]:
# Initialize query engine and run sample queries
query_engine = index.as_query_engine()  # use the vector db for queries

# Sample queries for testing
queries = [
    "Who won the Premier League in 2005?",
    "Describe the key players in Manchester United during the 2004-2005 season.",
    "Highlight the top goal scorers in the Premier League for the 2004-2005 season.",
    "How did Chelsea perform in the 2004-2005 football season?",
    "Provide an overview of the UEFA Champions League matches involving UK teams in 2004-2005.",
    "Detail the performance of Liverpool in the 2004-2005 season."
]

# Retry logic for queries
def query_with_retry(query_engine, query, retries=3, delay=5):
    for attempt in range(retries):
        try:
            response = query_engine.query(query)  # query the model with context
            return response.response
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                print("Retrying...")
                time.sleep(delay)
            else:
                raise

# Execute and print responses for each query
for query in queries:
    try:
        response = query_with_retry(query_engine, query)  # query the model with retry logic
        print(f"Query: {query}")
        print(f"Response: {response}\n")
    except Exception as e:
        print(f"Failed to get response for query '{query}': {e}")

Attempt 1 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 2 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 3 failed: [Errno 99] Cannot assign requested address
Failed to get response for query 'Who won the Premier League in 2005?': [Errno 99] Cannot assign requested address
Attempt 1 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 2 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 3 failed: [Errno 99] Cannot assign requested address
Failed to get response for query 'Describe the key players in Manchester United during the 2004-2005 season.': [Errno 99] Cannot assign requested address
Attempt 1 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 2 failed: [Errno 99] Cannot assign requested address
Retrying...
Attempt 3 failed: [Errno 99] Cannot assign requested address
Failed to get response for query 'Highlight the top goal scorers in the Premier League for the 2004-2005 seaso

KeyboardInterrupt: 

Inspect the output metadata

In [ ]:
response.metadata # print the full output from the LLM

{'fe4f5223-40c0-45a7-8660-65f2bc9c59cb': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'},
 '2e068bc6-8782-4cd6-aa8e-f6744af84203': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'}}